In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from datetime import datetime
import time
import numpy as np
import pandas as pd
import os

# from matplotlib import pyplot as plt
# import plotly.express as px
# %matplotlib inline

In [3]:
def create_year_halfs(years: list):
    '''Обычный ленивый скрипт для создания дат по формату американского времени для гугл трендов'''
    
    dates, sorted_years = [], sorted(years)
    quarters = ['-06-30', '-12-31']
    
    start_date = str(sorted_years[0]) + '-01-01' # добавляем стартовую точку для нащего периода
    
    for year in sorted_years: 
        quarters_year = [str(year) + date for date in quarters]
        dates.extend(quarters_year)
    
    dates.insert(0, start_date) # вставляем start_date в начало списка
    # например, если года 2007-2008, то вернет список ['2007-01-01', '2007-06-30', '2007-12-31', '2008-06-30', '2008-12-31']
    return dates


def download_csv(link: str, sleep_time=15, wait_button_time=120):
    '''Используем билиотеку selenium для автоматического открытия заданной ссылки
       Гугл трендов, робот нажимает на кнопку скачать и закрывает браузер'''
    
    
    # открываем драйвер гугла в этой же директории
    driver = webdriver.Chrome(executable_path='./chromedriver')
    time.sleep(5)
    
    # открываем гугл
    driver.get('https://www.google.ru')
    time.sleep(sleep_time)
    
    # открываем ссылку гугл трендов по нашему запросу в гугле
    driver.get(link)
    time.sleep(sleep_time)
    
    # ждем, пока кнопку можно будет нажать
    button = WebDriverWait(driver, wait_button_time).until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '.widget-actions-item.export')))
    
    # скачиваем файл
    button.click()
    time.sleep(5)
    
    # закрываем браузер
    driver.close()
    pass


def drop_days_columns(df: pd.DataFrame):
    # выдает список столбцов, которые содержат День_ и удаляет их из датафрейма
    df.rename(columns={'День_0': 'Дата'}, inplace=True)
    columns_to_drop = [column for column in df.columns.tolist() if 'День_' in column]
    df.drop(labels=columns_to_drop, axis=1, inplace=True)
    pass


def create_excel_region(ural_regions: dict, years: list, queries: list, path='/Users/ska19/Downloads'):
    ''' Принимает на вход словарь ural_regions: ключ название региона и его код в виде RU-XXX, 
        который будет вставляться в ссылку.
        Затем список годов years, которые Вам нужны для подкачки.
        И последне queries - слова, по которым вы хотите посмотреть поисковую активность.
        path - ваш путь к папке, куда по умолчанию скачиваются файлы.
        Данный скрипт скачивает ежедневные данные с сайта Гугл тренды, собирает их в датафрейм и затем сохраняет
        в виде эксель файла.'''
    

    dates = create_year_halfs(years) # создаем кварталы
    today = pd.Timestamp(datetime.today().date()) # дата на сегодня
    
    for key, value in ural_regions.items():  # идем по каждому региону
        df_region = pd.DataFrame()
        
        for i in range(0, len(queries), 5): # создаем запрос из 5 слов, это максимум для гугл трендов
            words = queries[i:i+5] # берем 5 слов
            query = ','.join(words)
            df_query = pd.DataFrame() # сюда заносим вывод на конкретный регион и запрос по кварталам
            
            
            for j in range(0, len(dates)-1):  # идем по полугодиям + 1 день назад из прошлого полугодия
                # нам нужно, что бы новое полугодие не выходило за рамки сегодняшней даты
                if today < pd.to_datetime(dates[j], yearfirst=True):
                    print(f"Date {dates[j]} is the future, today is {today}")
                    continue
                    
                # скачиваем файл по ссылке   
                link_to_open = f'https://trends.google.ru/trends/explore?date={dates[j]}%20{dates[j+1]}&geo={value}&q={query}'
                download_csv(link_to_open)
                
                # теперь работаем с самим файлом
                df_quarter = pd.read_csv(path + '/multiTimeline.csv', header=1) # открываем скачанный файл
                df_query = pd.concat([df_query, df_quarter], axis=0, ignore_index=True) # накладывает фремы друг на друга
                os.remove(path + '/multiTimeline.csv') # удаляем файл, чтобы программа могла прочитать следующий при подкачке
            
            day_column_rename = f'День_{i}' # переименовываем дни для теста
            df_query.rename(columns={'День': day_column_rename}, inplace=True)
            df_region.index = df_query.index # для метода join нужно, что бы индексы совпадали
            df_region = df_region.join(df_query) # добавляем новые запросы к старым по индексу (индексы должны совпадать XD)
        
        drop_days_columns(df_region) # лишние колонки с датой
        
        df_region.replace(to_replace='<1', value=0.5, inplace=True) # заменяем <1 для приведения столбцов к числовому формату
        df_region['Дата'] = pd.to_datetime(df_region['Дата']) # делаем формат даты
        columns_for_numeric = df_region.drop('Дата', axis=1).columns # колонки для преобразования к числовому формату
        df_region[columns_for_numeric] = df_region[columns_for_numeric].apply(pd.to_numeric) # преобразуем в числовой
        
        # df_region.to_csv(f'{key}_trends.csv') # можете сохранять в виде csv, если хотите
        df_region.to_excel(excel_writer=f'{key}_trends.xlsx', index=False) # сохраняем тренды региона в эксельку
        pass

In [8]:
years = list(range(2008, 2023)) 

ural_regions = {'Свердловская область': 'RU-SVE', 'Курганская область': 'RU-KGN', 'Тюменская область': 'RU-TYU',
                'ХМАО': 'RU-KHM', 'ЯНАО': 'RU-YAN', 'Челябинская область': 'RU-CHE', 'Республика Башкортостан':'RU-BA', 
                'Пермский край': 'RU-PER', 'Оренбургская область': 'RU-ORE'}

#ural_regions = {'Свердловская область': 'RU-SVE'}

queries = ['работа', 'вакансии', 'пособия по безработице']

In [104]:
create_excel_region(ural_regions, years, queries)

C:\Users\ska19\AppData\Local\Temp/ipykernel_6180/2478150312.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver')


Date 2022-06-30 is the future, today is 2022-06-08 00:00:00
Date 2022-06-30 is the future, today is 2022-06-08 00:00:00


In [9]:
ural_regions[0:4]

TypeError: unhashable type: 'slice'